<a href="https://colab.research.google.com/github/aleks-haksly/KarpovCources_Hard_DA_advanced/blob/main/03%20-%20Advanced%20experiments%20design/09_cuped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from statsmodels.stats.power import tt_ind_solve_power

In [ ]:
!wget https://raw.githubusercontent.com/aleks-haksly/AB_advanced/main/Part13%20-%20CUPED/Data/cuped_homework.csv -q

In [ ]:
data = pd.read_csv('/content/cuped_homework.csv')
data

,post_ARPPU,pre_ARPPU,group
0,660.0,595.0,A
1,540.0,621.0,A
2,863.0,782.0,A
3,431.0,567.0,A
4,434.0,473.0,A
...,...,...,...
4455,628.0,679.0,B
4456,758.0,634.0,B
4457,586.0,612.0,B
4458,698.0,791.0,B


## Задание 1. Первичный анализ (1/2) ##
Найдите наблюдаемую абсолютную разницу между средними у Б и А групп.

Введите значение в поле ниже с точностью до трех знаков в десятичной части.

In [ ]:
abs_diff = data.groupby('group').post_ARPPU.mean().diff()['B'].round(3)

In [ ]:
print(f"Yаблюдаемая абсолютная разница между средними у Б и А групп = {abs_diff}")

Yаблюдаемая абсолютная разница между средними у Б и А групп = 5.356


## Задание 1. Первичный анализ (2/2) ##
Укажите наблюдаемый effect size Cohen D. Для расчетов используйте абсолютную разницу и стандартное отклонение в pre данных в группе А.

Введите значение в поле ниже с точностью до трех знаков в десятичной части.

In [ ]:
cohen_d = (abs_diff / data[data.group == 'A'].pre_ARPPU.std()).round(3)
print(f"наблюдаемый effect size Cohen D = {cohen_d}")

наблюдаемый effect size Cohen D = 0.054


## Задание 2. Простой t-test ##
Найдите p-value, полученное при проведении простого t-test на post данных.

Введите значение в поле ниже с точностью до трех знаков в десятичной части.

In [ ]:
pval = ttest_ind(data.query('group == "A"').post_ARPPU, data.query('group == "B"').post_ARPPU, equal_var=False, alternative= 'two-sided').pvalue.round(3)

In [ ]:
print(f"p-value, полученное при проведении простого t-test на post данных = {pval}")

p-value, полученное при проведении простого t-test на post данных = 0.074


## Задание 3. CUPED t-test (1/2) ##
Найдите долю пустых значений в pre данных.

Введите значение в поле ниже с точностью до трех знаков в десятичной части.

In [ ]:
none_ratio = data.pre_ARPPU.isna().mean()

In [ ]:
print(f"доля пустых значений в pre данных = {none_ratio:.3f}")

доля пустых значений в pre данных = 0.133


# Задание 3. CUPED t-test (2/2) #
Найдите p-value в проведенном CUPED t-test. Предварительно заполните пустые значения средней на pre периоде.

Введите значение в поле ниже с точностью до трех знаков в десятичной части.

In [ ]:
data.pre_ARPPU.fillna(data.pre_ARPPU.mean(), inplace=True)

In [ ]:
gr_A = data.query('group == "A"')
gr_B = data.query('group == "B"')

In [ ]:
data.groupby('group').cov(numeric_only=True).reset_index()

,group,level_1,post_ARPPU,pre_ARPPU
0,A,post_ARPPU,9622.181424,5352.452033
1,A,pre_ARPPU,5352.452033,8424.748817
2,B,post_ARPPU,10454.025016,5860.881143
3,B,pre_ARPPU,5860.881143,9402.592218


In [ ]:
def ttest_theta(control_pre, control_post, test_pre, test_post):
  control_cov = np.cov(control_pre, control_post)
  test_cov = np.cov(test_pre, test_post)
  # с учетом того, что ковариация выборки самоу на себя есть ее дисперсия
  theta = (control_cov[0, 1] + test_cov[0, 1]) / (control_cov[0, 0] + test_cov[0, 0])
  return theta

def get_cuped_ttest(control_pre, control_post, test_pre, test_post):
    '''Проверяет гипотезу о равенстве средних CUPED вариант
    return - t_stat, p_value'''

    theta = ttest_theta(control_pre, control_post, test_pre, test_post)

    control_cuped = control_post - theta * control_pre
    test_cuped = test_post - theta * test_pre

    inference = ttest_ind(control_cuped, test_cuped)

    return(inference)

In [ ]:
control_pre = gr_A.pre_ARPPU
control_post = gr_A.post_ARPPU
test_pre = gr_B.pre_ARPPU
test_post = gr_B.post_ARPPU

In [ ]:
cuped_pval = get_cuped_ttest(control_pre, control_post, test_pre, test_post).pvalue.round(3)

In [ ]:
print(f"p-value в проведенном CUPED t-test = {cuped_pval}")

p-value в проведенном CUPED t-test = 0.032


## Задание 4. Размер выборки (1/5) ##
Параметры эксперимента для этого задания:

альфа = 0.05 и мощности 0.8

двусторонняя проверка гипотезы

размер выборок 50/50

статистический критерий t-test

относительный MDE = 0.01

В этом задании работаем с датасетом на исторических данных cuped_homework_pre_pre_data.csv. Он имеет похожую схему, только отсутствует деление на группы.

Не забудьте заполнить пустые значения средней, прежде чем рассчитывать размер группы для CUPED. Для расчета обычной выборки можно использовать post данные на историческом периоде.

In [ ]:
!wget https://raw.githubusercontent.com/aleks-haksly/AB_advanced/main/Part13%20-%20CUPED/Data/cuped_homework_pre_pre_data.csv?token=GHSAT0AAAAAACTJMDQNTFRHS2XM54FPFI6MZUM3FKA

--2024-07-09 06:30:58--  https://raw.githubusercontent.com/aleks-haksly/AB_advanced/main/Part13%20-%20CUPED/Data/cuped_homework_pre_pre_data.csv?token=GHSAT0AAAAAACTJMDQNTFRHS2XM54FPFI6MZUM3FKA
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80352 (78K) [text/plain]
Saving to: ‘cuped_homework_pre_pre_data.csv?token=GHSAT0AAAAAACTJMDQNTFRHS2XM54FPFI6MZUM3FKA’

cuped_homework_pre_ 100%[===================>]  78.47K  --.-KB/s    in 0.08s   

2024-07-09 06:30:59 (977 KB/s) - ‘cuped_homework_pre_pre_data.csv?token=GHSAT0AAAAAACTJMDQNTFRHS2XM54FPFI6MZUM3FKA’ saved [80352/80352]



In [ ]:
data = pd.read_csv("/content/cuped_homework_pre_pre_data.csv?token=GHSAT0AAAAAACTJMDQNTFRHS2XM54FPFI6MZUM3FKA")

In [ ]:
data.head()

,post_ARPPU,pre_ARPPU
0,629.0,620.0
1,536.0,619.0
2,503.0,596.0
3,649.0,578.0
4,667.0,752.0


In [ ]:
alpha = 0.05
power = 0.8
alternative = 'two-sided'
ratio = 1
MDE = 0.01

In [ ]:
data.isna().sum()

post_ARPPU    0
pre_ARPPU     0
dtype: int64

In [ ]:
data.pre_ARPPU.fillna(data.pre_ARPPU.mean(), inplace=True)

## Задание 4. Размер выборки (2/5) ##
Укажите, чему равна theta по вашим расчетам.

Введите значение в поле ниже с точностью до трех знаков в десятичной части.

In [ ]:
theta = data.cov(numeric_only=True)["pre_ARPPU"].rolling(2).apply(lambda x: x[0]/ x[1]).iloc[1].round(3)
theta

0.61

## Задание 4. Размер выборки (3/5) ##
Укажите, на сколько процентов стандартное отклонение Y_cuped меньше стандартного отклонения post_ARPPU.

Введите значение в процентах в поле ниже с точностью до одного знака в десятичной части.

In [ ]:
data.post_ARPPU.std()

101.24946625472516

In [ ]:
data["Y_cuped"] = data.post_ARPPU - theta * data.pre_ARPPU

In [ ]:
diff = (data.post_ARPPU.std() - data.Y_cuped.std()) / data.post_ARPPU.std()

In [ ]:
print(f"Y_cuped меньше стандартного отклонения post_ARPPU на {diff:.1%}")

Y_cuped меньше стандартного отклонения post_ARPPU на 17.6%


## Задание 4. Размер выборки (4/5) ##
Укажите размер выборки, необходимый для детекции такого MDE на post данных простым t-test.

In [ ]:
uplift = MDE * data.post_ARPPU.mean()
cohen_D = uplift / data.post_ARPPU.std()

In [ ]:
sample_size = np.ceil(tt_ind_solve_power(effect_size = cohen_D, alpha=alpha, power=power, ratio=ratio, alternative=alternative))

In [ ]:
print(f"размер выборки, необходимый для детекции такого MDE на post данных простым t-test равен {sample_size}")

размер выборки, необходимый для детекции такого MDE на post данных простым t-test равен 4474.0


## Задание 4. Размер выборки (5/5) ##
Укажите размер выборки, необходимый для детекции такого MDE CUPED t-test.

In [ ]:
data.Y_cuped.std()

83.47181518345718

In [ ]:
cohen_D_cuped = uplift / data.Y_cuped.std()

In [ ]:
sample_size = np.ceil(tt_ind_solve_power(effect_size = cohen_D_cuped, alpha=alpha, power=power, ratio=ratio, alternative=alternative))

In [ ]:
print(f"размер выборки, необходимый для детекции такого MDE CUPED t-test равен {sample_size}")

размер выборки, необходимый для детекции такого MDE CUPED t-test равен 3041.0
